In [ ]:
import os.path

import pandas as pd

import vjp.preprocess as preprocess

Retrieve a preprocessed dataframe using the pipeline defined by `data_exploration.ipynb`.

In [ ]:
# Load from file if exists (e.g. pregenerated via preprocess CLI)
DF_FILENAME = 'connected_components.parquet'
if os.path.exists(DF_FILENAME):
    print(f'Reading from {DF_FILENAME}...')
    df = pd.read_parquet(DF_FILENAME)
else:           # Generate on the go
    print('File not found, generating dataframe...')
    namespace = preprocess.Namespace()
    namespace.connected_component_tags = ('req', 'arg', 'claim', 'mot', 'dec')
    namespace.level = preprocess.PreprocessingLevels.CONNECTED_COMPONENTS
    df = preprocess.preprocess(namespace)

df.head()

All tag types are gathered, so that multiple experiments may be carried out by excluding some of the columns.